This notebook runs the econometric and machine learning models using a shorter coverage period from 2015 to 2025. This is done to assess the robustness of the actual model results and account for inconsistencies between GDELT 1.0 and 2.1 data versions. Thr link to the dataset on which this robustness check has been performed can be found here: https://drive.google.com/file/d/16vUg47ofFbm1e6U1W4r6RTcwxS2W5Fy8/view?usp=sharing

In [ ]:
# Importing all the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler


#Load the dataset into the data frame
df1=pd.read_csv('/content/Robustness check_2015-2025.csv')

In [ ]:
df1

,RegionName,AreaCode,AveragePrice,HPI,SalesVolume,AW_Regular_Earnings,UnemploymentRate,CPI,MortgageApprovals,MortgageRate,...,ConstructionCost_Index,GT_demand,GT_economicpolicy,GT_marketawareness,GT_mortgage_financing,GT_renting_affordability,GDELT_average_news_tone,Source,GDELT_Version,GDELT_average_news_tone_normalized
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,Aberdeenshire,S12000034,207121,100.7,372,448.236429,5.6,0.5,58916,2.01,...,100.5,-1.302967,-0.49833,-1.116325,0.728186,1.309258,2.742955,Events_v1_0_ECON,GDELT 1.0,-0.029375
2015-01-01,Swansea,W06000011,128592,66.1,195,448.236429,5.6,0.5,58916,2.01,...,100.5,-1.302967,-0.49833,-1.116325,0.728186,1.309258,2.742955,Events_v1_0_ECON,GDELT 1.0,-0.029375
2015-01-01,Chichester,E07000225,328444,73.6,120,448.236429,5.6,0.5,58916,2.01,...,100.5,-1.302967,-0.49833,-1.116325,0.728186,1.309258,2.742955,Events_v1_0_ECON,GDELT 1.0,-0.029375
2015-01-01,Swale,E07000113,187465,61.5,167,448.236429,5.6,0.5,58916,2.01,...,100.5,-1.302967,-0.49833,-1.116325,0.728186,1.309258,2.742955,Events_v1_0_ECON,GDELT 1.0,-0.029375
2015-01-01,Chorley,E07000118,134089,66.9,112,448.236429,5.6,0.5,58916,2.01,...,100.5,-1.302967,-0.49833,-1.116325,0.728186,1.309258,2.742955,Events_v1_0_ECON,GDELT 1.0,-0.029375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-01,Rochford,E07000075,393404,94.5,30,678.812393,4.7,4.1,64571,4.32,...,152.4,-0.326596,0.48053,-0.332495,-0.639135,0.300443,-0.826562,GKG_v2_0_Housing,GDELT 2.0,1.371211
2025-06-01,Enfield,E09000010,480102,100.4,56,678.812393,4.7,4.1,64571,4.32,...,152.4,-0.326596,0.48053,-0.332495,-0.639135,0.300443,-0.826562,GKG_v2_0_Housing,GDELT 2.0,1.371211
2025-06-01,Stockport,E08000007,308294,105.0,126,678.812393,4.7,4.1,64571,4.32,...,152.4,-0.326596,0.48053,-0.332495,-0.639135,0.300443,-0.826562,GKG_v2_0_Housing,GDELT 2.0,1.371211


Cleaning dates

In [ ]:
# Identifying and converting the date column to datetime
date_col = "Date"
df1[date_col] = pd.to_datetime(df1[date_col], errors="coerce")

# Sorting by date and dropping null values
df1 = df1.sort_values(by=date_col).dropna(subset=[date_col])
df1 = df1.reset_index(drop=True)

# Setting the date as index for time-series analysis
df1 = df1.set_index(date_col)
print(df1.index.min(), "→", df1.index.max())


2015-01-01 00:00:00 → 2025-06-01 00:00:00


Checking for missing values

In [ ]:
# Missing value analysis
# Checking the total number of missing values for each variable
missing_summary = df1.isnull().sum().sort_values(ascending=False)
print("Missing values per column:\n")
print(missing_summary[missing_summary > 0])

# Percentage of missing values
missing_percent = (df1.isnull().sum() / len(df1)) * 100
print("\nPercentage missing per column:\n")
print(missing_percent[missing_percent > 0].round(2))




Missing values per column:

BM_New Housing    2025
dtype: int64

Percentage missing per column:

BM_New Housing    3.97
dtype: float64


In [ ]:
# Filter rows where BM_New Housing is missing
missing_bm = df1[df1['BM_New Housing'].isnull()]

# Display the date range of missing values
print("First 10 missing entries:")
print(missing_bm.index.to_series().head(10))
1
# Count missing by year
missing_by_year = missing_bm.groupby(missing_bm.index.year).size()
print("\nMissing count by year:")
print(missing_by_year)


First 10 missing entries:
Date
2025-02-01   2025-02-01
2025-02-01   2025-02-01
2025-02-01   2025-02-01
2025-02-01   2025-02-01
2025-02-01   2025-02-01
2025-02-01   2025-02-01
2025-02-01   2025-02-01
2025-02-01   2025-02-01
2025-02-01   2025-02-01
2025-02-01   2025-02-01
Name: Date, dtype: datetime64[ns]

Missing count by year:
Date
2025    2025
dtype: int64


In [ ]:
#Correcting for Missingness
# Forward-filling the BM_New Housing variable using January 2025's value
df1['BM_New Housing'] = df1['BM_New Housing'].fillna(method='ffill')

# Checking if any missing values still remain for the affected period February 2025 to June 2025
print("Remaining missing:", df1['BM_New Housing'].isnull().sum())
print(df1.loc['2025-01-01':'2025-06-01', ['BM_New Housing']])

Remaining missing: 0
            BM_New Housing
Date                      
2025-01-01           152.4
2025-01-01           152.4
2025-01-01           152.4
2025-01-01           152.4
2025-01-01           152.4
...                    ...
2025-06-01           152.4
2025-06-01           152.4
2025-06-01           152.4
2025-06-01           152.4
2025-06-01           152.4

[2430 rows x 1 columns]


/tmp/ipython-input-2119981912.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df1['BM_New Housing'] = df1['BM_New Housing'].fillna(method='ffill')


Renaming Variables

In [ ]:
# Renaming the variables
df1.rename(columns={
    'Index': 'HPI',
    'AWE_Regular': 'AW_Regular_Earnings',
    'MortgageRate_2YFix': 'MortgageRate',
    'AvgTone_Stitched': 'GDELT_average_news_tone',
    'gt_trend_buying_demand': 'GT_demand',
    'gt_trend_economic_policy': 'GT_economicpolicy',
    'gt_trend_market_awareness': 'GT_marketawareness',
    'gt_trend_mortgage_financing': 'GT_mortgage_financing',
    'gt_trend_renting_affordability': 'GT_renting_affordability',
    'BM_New Housing': 'ConstructionCost_Index'
}, inplace=True)

# Inspecting the renamed column names
df1.columns


Index(['RegionName', 'AreaCode', 'AveragePrice', 'HPI', 'SalesVolume',
       'AWE_Total', 'AW_Regular_Earnings', 'UnemploymentRate', 'CPI',
       'MortgageApprovals', 'MortgageRate', 'BankRate', 'ConsumerConfidence',
       'ConstructionCost_Index', 'GT_demand', 'GT_economicpolicy',
       'GT_marketawareness', 'GT_mortgage_financing',
       'GT_renting_affordability', 'GDELT_average_news_tone', 'Docs_Stitched',
       'Source'],
      dtype='object')

In [ ]:
# Creating a new categorical variable for GDELT version to account for the transition from GDELT 1.0 to GDELT 2.0
df1['GDELT_Version'] = np.where(df1.index < '2015-02-01', 'GDELT 1.0', 'GDELT 2.0')



In [ ]:
# Normalisation of GDELT average news tone for version 1.0 and 2.0

df1['GDELT_average_news_tone_normalized'] = (
    df1.groupby('GDELT_Version')['GDELT_average_news_tone']
      .transform(lambda x: (x - x.mean()) / x.std())
)


In [ ]:
# Standardising the macroeconomic predictors before modelling
macro_features = [
    'AW_Regular_Earnings','UnemploymentRate', 'CPI', 'MortgageApprovals', 'MortgageRate','BankRate', 'ConsumerConfidence', 'ConstructionCost_Index'
]

scaler = StandardScaler()
df1_scaled = df1.copy()
df1_scaled[macro_features] = scaler.fit_transform(df1_scaled[macro_features])

df1_scaled.head()

,RegionName,AreaCode,AveragePrice,HPI,SalesVolume,AWE_Total,AW_Regular_Earnings,UnemploymentRate,CPI,MortgageApprovals,...,GT_demand,GT_economicpolicy,GT_marketawareness,GT_mortgage_financing,GT_renting_affordability,GDELT_average_news_tone,Docs_Stitched,Source,GDELT_Version,GDELT_average_news_tone_normalized
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,Aberdeenshire,S12000034,207121,100.7,372,475.611058,-1.255088,2.386032,-1.008356,-0.491529,...,-1.302967,-0.49833,-1.116325,0.728186,1.309258,2.742955,60775,Events_v1_0_ECON,GDELT 1.0,-0.029375
2015-01-01,Swansea,W06000011,128592,66.1,195,475.611058,-1.255088,2.386032,-1.008356,-0.491529,...,-1.302967,-0.49833,-1.116325,0.728186,1.309258,2.742955,60775,Events_v1_0_ECON,GDELT 1.0,-0.029375
2015-01-01,Chichester,E07000225,328444,73.6,120,475.611058,-1.255088,2.386032,-1.008356,-0.491529,...,-1.302967,-0.49833,-1.116325,0.728186,1.309258,2.742955,60775,Events_v1_0_ECON,GDELT 1.0,-0.029375
2015-01-01,Swale,E07000113,187465,61.5,167,475.611058,-1.255088,2.386032,-1.008356,-0.491529,...,-1.302967,-0.49833,-1.116325,0.728186,1.309258,2.742955,60775,Events_v1_0_ECON,GDELT 1.0,-0.029375
2015-01-01,Chorley,E07000118,134089,66.9,112,475.611058,-1.255088,2.386032,-1.008356,-0.491529,...,-1.302967,-0.49833,-1.116325,0.728186,1.309258,2.742955,60775,Events_v1_0_ECON,GDELT 1.0,-0.029375


In [ ]:
# Cleaning the formatting issues in the region names
df1['RegionName'] = (
    df1['RegionName']
    .str.strip()
    .str.title()
)

# Correcting specific cases where title-casing produces incorrect results
df1['RegionName'] = df1['RegionName'].replace({
    "King'S Lynn And West Norfolk": "King's Lynn and West Norfolk",
    "Na H-Eileanan Siar": "Na h-Eileanan Siar"
})

In [ ]:
# Dropping unnecessary variables
vars_to_drop = [
    'AWE_Total',
    'Docs_Stitched'
]

df1 = df1.drop(columns=vars_to_drop, errors='ignore')

In [ ]:
# Applying transformations to non-stationary variables to ensure stationarity
from statsmodels.tsa.stattools import adfuller

uk_df = df1[df1['RegionName'] == 'United Kingdom'].copy().sort_index()

# Variables to test and transform
vars_to_test = [
    'HPI', 'AveragePrice', 'SalesVolume',
    'AW_Regular_Earnings',
    'UnemploymentRate', 'CPI', 'MortgageApprovals',
    'MortgageRate', 'BankRate',
    'ConsumerConfidence', 'ConstructionCost_Index',
    'GT_demand', 'GT_economicpolicy', 'GT_marketawareness',
    'GT_mortgage_financing', 'GT_renting_affordability',
    'GDELT_average_news_tone_normalized'
]

def adf_pvalue(series, autolag="AIC"):
    """Run a simple ADF test and return the p-value (or NaN if too short)."""
    s = series.dropna()
    if len(s) < 24:
        return np.nan
    stat, pval, *_ = adfuller(s, autolag=autolag, regression="c")
    return pval

def choose_transform(s: pd.Series, seasonal_lag: int = 12):
    """
    Try, in this order:
      1. level
      2. log-diff (if strictly positive)
      3. first difference
      4. add seasonal diff on top of the chosen diff/logdiff
    Return: transformed_series, label, p_final, dict of all attempted p-values.
    """
    attempts = {}

    # 1) Level
    p_level = adf_pvalue(s)
    attempts["level"] = p_level
    if p_level < 0.05:
        return s, "level", p_level, attempts

    # Base transformation: log-diff if possible, otherwise first difference
    if (s > 0).all():
        base = np.log(s).diff()
        base_label = "logdiff"
    else:
        base = s.diff()
        base_label = "diff"

    p_base = adf_pvalue(base)
    attempts[base_label] = p_base
    if p_base < 0.05:
        return base, base_label, p_base, attempts

    # 3) Seasonal diff on top of base, if we have enough data
    if seasonal_lag is not None and len(base.dropna()) > seasonal_lag + 12:
        seas = base - base.shift(seasonal_lag)
        p_seas = adf_pvalue(seas)
        label_seas = base_label + "+seas"
        attempts[label_seas] = p_seas
        if p_seas < 0.05:
            return seas, label_seas, p_seas, attempts

    # 4) If nothing is stationary at 5%, pick the attempt with the lowest p-value
    #    and flag it as non-stationary.
    valid_attempts = {k: v for k, v in attempts.items() if not pd.isna(v)}
    if valid_attempts:
        best_label = min(valid_attempts, key=valid_attempts.get)
        best_p = valid_attempts[best_label]
    else:
        best_label, best_p = base_label, p_base

    # Choose the corresponding series
    if best_label == "level":
        best_series = s
    elif best_label == "logdiff":
        best_series = base  # logdiff
    elif best_label == "diff":
        best_series = base  # first diff
    else:  # seasonal version
        best_series = base - base.shift(seasonal_lag)

    return best_series, best_label + " (non-stationary, lowest p)", best_p, attempts

# Apply to all variables
transformed = {}
summary_rows = []

for v in vars_to_test:
    if v not in uk_df.columns:
        continue

    series = uk_df[v].astype(float)

    p_level = adf_pvalue(series)
    s_trans, label, p_final, attempts = choose_transform(series)

    transformed[v + "_TRANS"] = s_trans

    # Pack up a short notes string with attempted p-values
    notes_str = "; ".join(
        f"{k}: p={attempts[k]:.4g}" for k in attempts if not pd.isna(attempts[k])
    )

    summary_rows.append({
        "Variable": v,
        "ADF p (level)": p_level,
        "Chosen transform": label,
        "ADF p (final)": p_final,
        "Notes": notes_str
    })

adf_summary = pd.DataFrame(summary_rows).sort_values("ADF p (final)")
uk_stationary = pd.DataFrame(transformed, index=uk_df.index).dropna(how="all")

print("ADF-based transformation completed.")
print(f"uk_stationary shape: {uk_stationary.shape}")
display(adf_summary)


ADF-based transformation completed.
uk_stationary shape: (126, 17)


,Variable,ADF p (level),Chosen transform,ADF p (final),Notes
9,ConsumerConfidence,3.961916e-19,level,3.961916e-19,level: p=3.962e-19
12,GT_economicpolicy,4.563452e-01,diff,1.393864e-18,level: p=0.4563; diff: p=1.394e-18
8,BankRate,2.990156e-01,logdiff,2.356632e-18,level: p=0.299; logdiff: p=2.357e-18
7,MortgageRate,5.759688e-01,logdiff,6.935283e-09,level: p=0.576; logdiff: p=6.935e-09
10,ConstructionCost_Index,6.288087e-01,logdiff,1.201339e-07,level: p=0.6288; logdiff: p=1.201e-07
3,AW_Regular_Earnings,9.990879e-01,logdiff,6.081721e-06,level: p=0.9991; logdiff: p=6.082e-06
2,SalesVolume,6.066230e-01,logdiff,3.866614e-04,level: p=0.6066; logdiff: p=0.0003867
16,GDELT_average_news_tone_normalized,1.759964e-03,level,1.759964e-03,level: p=0.00176
1,AveragePrice,8.404472e-01,logdiff+seas,3.787519e-03,level: p=0.8404; logdiff: p=0.3055; logdiff+se...
0,HPI,8.425152e-01,logdiff+seas,4.738522e-03,level: p=0.8425; logdiff: p=0.3128; logdiff+se...


In [ ]:
# Regional Panel dataset for ML models

# Load and prepare base dataframe
df = df1.copy()

# Ensure Date column exists
if isinstance(df.index, pd.DatetimeIndex):
    df = df.reset_index().rename(columns={'index': 'Date'})
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['RegionName', 'Date']).reset_index(drop=True)

# Transformation map (In the earlier section, I had already determined the transformations needed for each variable)
TRANSFORM_MAP = {
    'HPI': 'logdiff',
    'AveragePrice': 'logdiff',
    'SalesVolume': 'logdiff',
    'AW_Regular_Earnings': 'logdiff_seas',
    'UnemploymentRate': 'logdiff',
    'CPI': 'logdiff',
    'MortgageApprovals': 'logdiff',
    'MortgageRate': 'logdiff',
    'BankRate': 'logdiff',
    'ConsumerConfidence': 'level',
    'ConstructionCost_Index': 'logdiff_seas',
    'GT_demand': 'diff',
    'GT_economicpolicy': 'diff',
    'GT_marketawareness': 'diff',
    'GT_mortgage_financing': 'diff',
    'GT_renting_affordability': 'diff',
    'GDELT_average_news_tone_normalized': 'level'
}

def apply_transform(series, method):
    if method == 'logdiff':
        return np.log(series).replace([-np.inf, np.inf], np.nan).diff()
    elif method == 'diff':
        return series.diff()
    elif method == 'logdiff_seas':
        x = np.log(series).replace([-np.inf, np.inf], np.nan).diff()
        return x.diff(12)
    elif method == 'level':
        return series
    else:
        raise ValueError(f"Unknown method: {method}")

# Applying the transformations
panel = df[['Date','RegionName'] + list(TRANSFORM_MAP.keys())].copy()
for col, how in TRANSFORM_MAP.items():
    panel[f"{col}_TRANS"] = (
        panel.groupby('RegionName', group_keys=False)[col]
             .apply(apply_transform, method=how)
    )

TARGET = "HPI_TRANS"
trans_cols = [f"{c}_TRANS" for c in TRANSFORM_MAP.keys()]
panel_trans = panel[['Date','RegionName'] + trans_cols].copy()

# Create 1–3 lags
MAX_LAG = 3
pred_cols = [c for c in panel_trans.columns if c.endswith('_TRANS') and c != TARGET]

def make_lags(g):
    g = g.sort_values('Date').copy()
    for col in pred_cols + [TARGET]:
        for L in range(1, MAX_LAG + 1):
            g[f"{col}_lag{L}"] = g[col].shift(L)
    return g

panel_lagged = panel_trans.groupby('RegionName', group_keys=False).apply(make_lags)

# Final unscaled modelling dataset
lag_cols = [c for c in panel_lagged.columns if "_lag" in c]
final_ml = (
    panel_lagged[['Date','RegionName', TARGET] + lag_cols]
    .dropna()
    .reset_index(drop=True)
)

print("Regional ML dataset created:", final_ml.shape)
final_ml.to_csv("regional_ml_nowcast_features_lag1to3.csv", index=False)

/tmp/ipython-input-1354428189.py:69: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  panel_lagged = panel_trans.groupby('RegionName', group_keys=False).apply(make_lags)


Regional ML dataset created: (44550, 54)


In [ ]:
#Constructing traditional, high-frequency, and combined feature sets from existing lagged variables

traditional_vars = [
    'AW_Regular_Earnings_TRANS', 'UnemploymentRate_TRANS',
    'CPI_TRANS', 'MortgageApprovals_TRANS', 'MortgageRate_TRANS',
    'BankRate_TRANS', 'ConsumerConfidence_TRANS', 'ConstructionCost_Index_TRANS'
]

hf_vars = [
    'GT_demand_TRANS', 'GT_economicpolicy_TRANS', 'GT_marketawareness_TRANS',
    'GT_mortgage_financing_TRANS', 'GT_renting_affordability_TRANS',
    'GDELT_average_news_tone_normalized_TRANS'
]

combined_vars = traditional_vars + hf_vars


def get_existing_lags(var_list):
    return [col for col in final_ml.columns if any(col.startswith(v + "_lag") for v in var_list)]

FEATURE_SETS = {
    "traditional": get_existing_lags(traditional_vars),
    "highfreq": get_existing_lags(hf_vars),
    "combined": get_existing_lags(combined_vars)
}

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

#Evaluation metrics-RMSE and MAE
def eval_metrics(y_true, y_pred):
    y_true, y_pred = np.asarray(y_true), np.asarray(y_pred)

    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae  = mean_absolute_error(y_true, y_pred)

    return rmse, mae

Econometric Models

ARIMA

In [ ]:
import warnings
import itertools
from statsmodels.tsa.arima.model import ARIMA

# Using the stationary HPI series
y = uk_stationary['HPI_TRANS'].dropna()

p_range = range(0, 6)
q_range = range(0, 6)

candidate_orders = list(itertools.product(p_range, [0], q_range))

results = []

# Trying each (p,0,q) combination and record its AIC/BIC
for order in candidate_orders:
    try:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            model = ARIMA(y, order=order).fit()

        results.append({
            "order": order,
            "AIC": model.aic,
            "BIC": model.bic
        })
    except:
        # Some specifications may fail to converge; skip quietly
        continue

# Converting results to a DataFrame and sort by AIC
order_ranked = (
    pd.DataFrame(results)
    .sort_values("AIC")
    .reset_index(drop=True)
)

print("Top 5 ARIMA(p,0,q) models ranked by AIC:")
print(order_ranked.head(5))

Top 5 ARIMA(p,0,q) models ranked by AIC:
       order         AIC         BIC
0  (1, 0, 3) -709.496987 -693.132660
1  (0, 0, 5) -707.053149 -687.961435
2  (0, 0, 4) -706.997103 -690.632776
3  (1, 0, 4) -706.419373 -687.327658
4  (1, 0, 5) -705.308533 -683.489431


In [ ]:
# ARIMA (1,0,3)
import time

# Preparing the stationary UK HPI series
y = uk_stationary['HPI_TRANS'].dropna().copy()
y.index = pd.to_datetime(y.index).to_period('M').to_timestamp('M')
y = y.sort_index()

START_TEST = pd.Timestamp('2018-01-31').to_period('M').to_timestamp('M')
best_order = (1, 0, 3)   # chosen earlier using AIC/BIC

# Containers for forecasts, actual values and timing
preds, actuals, dates = [], [], []
train_times, pred_times = [], []

# Rolling expanding-window forecast with timing
t0 = time.time()

for t in y.index[y.index >= START_TEST]:

    # Use all data up to the previous month as the training sample
    train_end = (t.to_period('M') - 1).to_timestamp('M')
    train = y.loc[:train_end]
    if len(train) < 36:
        continue

    # Time the estimation step
    t_train_start = time.time()
    model = ARIMA(train, order=best_order).fit()
    t_train_end = time.time()
    train_times.append(t_train_end - t_train_start)

    # Time the prediction step
    t_pred_start = time.time()
    pred = float(model.forecast(1).iloc[0])
    t_pred_end = time.time()
    pred_times.append(t_pred_end - t_pred_start)

    # Store forecast and realised value
    preds.append(pred)
    actuals.append(float(y.loc[t]))
    dates.append(t)

# Forecast accuracy (RMSE and MAE)
rmse, mae = eval_metrics(actuals, preds)
print(f"[ARIMA{best_order}] Steps={len(preds)}  RMSE={rmse:.5f}  MAE={mae:.5f}")

# Computational efficiency summary
total_time = time.time() - t0

avg_train = float(np.mean(train_times)) if train_times else np.nan
avg_pred  = float(np.mean(pred_times)) if pred_times else np.nan
avg_total = avg_train + avg_pred if train_times and pred_times else np.nan

print("\nARIMA computational efficiency (expanding window):")
print(f"Total time for rolling forecasts: {total_time:.2f} seconds")
print(f"Average training time per forecast:   {avg_train:.4f} seconds")
print(f"Average prediction time per forecast: {avg_pred:.6f} seconds")
print(f"Average total time per forecast:      {avg_total:.4f} seconds")

# Save actual vs predicted series
arima_nowcast = pd.DataFrame(
    {"ARIMA_pred": preds, "HPI_TRANS": actuals},
    index=pd.Index(dates, name="Date")
)

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stat

[ARIMA(1, 0, 3)] Steps=77  RMSE=0.01370  MAE=0.00987

ARIMA computational efficiency (expanding window):
Total time for rolling forecasts: 27.12 seconds
Average training time per forecast:   0.3437 seconds
Average prediction time per forecast: 0.008005 seconds
Average total time per forecast:      0.3517 seconds


/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


SARIMA

In [ ]:
import numpy as np
from itertools import product
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Preparing stationary UK HPI series
y = uk_stationary['HPI_TRANS'].dropna().copy()
y.index = pd.to_datetime(y.index).to_period('M').to_timestamp('M')
y = y.sort_index()

# Using only data before 2018 for order selection
START_TEST = pd.Timestamp('2018-01-31').to_period('M').to_timestamp('M')
y_train = y.loc[y.index < START_TEST]

print("SARIMA order selection sample:")
print(y_train.index.min(), "→", y_train.index.max())
print("Number of observations:", len(y_train))

# Grid of candidate (p,d,q) and seasonal (P,D,Q,12)
orders = [(p, 0, q) for p in range(0, 4) for q in range(0, 4)]   # d = 0 (already differenced)
seasonals = [(P, D, Q, 12) for P in (0, 1) for D in (0, 1) for Q in (0, 1)]

best_aic = np.inf
best_order = None
best_seasonal = None

for order, seas in product(orders, seasonals):
    try:
        model = SARIMAX(
            y_train,
            order=order,
            seasonal_order=seas,
            enforce_stationarity=False,
            enforce_invertibility=False
        ).fit(disp=False)

        if model.aic < best_aic:
            best_aic = model.aic
            best_order = order
            best_seasonal = seas
    except Exception:
        continue

print(f"Best SARIMA order (pre-2018, by AIC): order={best_order}, seasonal={best_seasonal}, AIC={best_aic:.2f}")

SARIMA order selection sample:
2016-02-29 00:00:00 → 2017-12-31 00:00:00
Number of observations: 23


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for va

Best SARIMA order (pre-2018, by AIC): order=(0, 0, 0), seasonal=(0, 0, 0, 12), AIC=-165.24


/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


In [ ]:
# SARIMA model
import time

# Using the same stationary HPI series
y = uk_stationary['HPI_TRANS'].dropna().copy()
y.index = pd.to_datetime(y.index).to_period('M').to_timestamp('M')
y = y.sort_index()

START_TEST = pd.Timestamp('2018-01-31').to_period('M').to_timestamp('M')

# Using the SARIMA specification chosen above
sarima_order = best_order
sarima_seasonal = best_seasonal

preds, actuals, dates = [], [], []
train_times, pred_times = [], []

t0 = time.time()  # total rolling time

# Expanding-window one-step-ahead forecasts
for t in y.index[y.index >= START_TEST]:

    # Training sample: all data up to the previous month
    train_end = (t.to_period('M') - 1).to_timestamp('M')
    train = y.loc[:train_end]

    if len(train) < 36:
        continue

    try:
        # Time the model estimation
        t_train_start = time.time()
        model = SARIMAX(
            train,
            order=sarima_order,
            seasonal_order=sarima_seasonal,
            enforce_stationarity=False,
            enforce_invertibility=False
        ).fit(disp=False)
        t_train_end = time.time()
        train_times.append(t_train_end - t_train_start)

        # Time the one-step-ahead prediction
        t_pred_start = time.time()
        forecast = float(model.forecast(1).iloc[0])
        t_pred_end = time.time()
        pred_times.append(t_pred_end - t_pred_start)

        # Store forecast and actual
        preds.append(forecast)
        actuals.append(float(y.loc[t]))
        dates.append(t)

    except Exception:
        continue

total_time = time.time() - t0

# Evaluation metrics for forecast accuracy
rmse, mae = eval_metrics(actuals, preds)
print(f"\n[SARIMA {sarima_order}, seasonal={sarima_seasonal}] "
      f"Steps={len(preds)}  RMSE={rmse:.5f}  MAE={mae:.5f}")

# Computational efficiency summary
avg_train = float(np.mean(train_times)) if train_times else np.nan
avg_pred  = float(np.mean(pred_times)) if pred_times else np.nan
avg_total = avg_train + avg_pred if train_times and pred_times else np.nan

print("\nSARIMA computational efficiency (expanding window):")
print(f"Total rolling nowcast time: {total_time:.2f} seconds")
print(f"Average training time per forecast:   {avg_train:.4f} seconds")
print(f"Average prediction time per forecast: {avg_pred:.6f} seconds")
print(f"Average total time per forecast:      {avg_total:.4f} seconds")

# Saving prediction vs actual series
sarima_nowcast = pd.DataFrame(
    {"SARIMA_pred": preds, "HPI_TRANS": actuals},
    index=pd.Index(dates, name="Date")
)

/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



[SARIMA (0, 0, 0), seasonal=(0, 0, 0, 12)] Steps=77  RMSE=0.01603  MAE=0.01059

SARIMA computational efficiency (expanding window):
Total rolling nowcast time: 1.82 seconds
Average training time per forecast:   0.0194 seconds
Average prediction time per forecast: 0.003819 seconds
Average total time per forecast:      0.0232 seconds


VAR

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR

# Base VAR specification
VAR_SPECS = {
    "traditional": traditional_vars,
    "highfreq": hf_vars,
    "combined": traditional_vars + hf_vars
}

def build_var_dataset(uk_stationary, predictor_list, corr_thresh=0.98):
    """
    Build a VAR dataset with HPI_TRANS + chosen predictors.
    - keeps only columns that exist
    - drops missing rows
    - enforces month-end DateTimeIndex
    - prunes extremely collinear predictors (|corr| > corr_thresh)
    """
    cols = ["HPI_TRANS"] + [c for c in predictor_list if c in uk_stationary.columns]
    Z = uk_stationary[cols].dropna().astype(float).copy()

    # Month-end index (consistent with your other code)
    Z.index = pd.to_datetime(Z.index).to_period("M").to_timestamp("M")
    Z = Z.sort_index()

    # Prune near-duplicate predictors (optional)
    predictors_only = [c for c in Z.columns if c != "HPI_TRANS"]
    if len(predictors_only) >= 2:
        corr = Z[predictors_only].corr().abs()
        to_drop = set()
        for i in range(len(corr.columns)):
            for j in range(i):
                if corr.iloc[i, j] > corr_thresh:
                    to_drop.add(corr.columns[i])

        if to_drop:
            Z = Z.drop(columns=list(to_drop))
            print("Dropped highly collinear predictors:", sorted(to_drop))

    return Z


def feasible_maxlags(T, k, hard_cap):
    """
    Conservative feasibility cap for VAR lag order.
    Rule-of-thumb: floor((T - 1) / k) - 1, then clamp to [1, hard_cap].
    If that goes below 1, return 1 (we'll fallback to p=1).
    """
    p_cap = int((T - 1) // max(k, 1) - 1)
    p_cap = min(p_cap, hard_cap)
    return max(1, p_cap)


# Store chosen lag length per spec
VAR_LAG_ORDERS = {}

START_TEST = pd.Timestamp("2018-01-31").to_period("M").to_timestamp("M")

MAX_LAGS = 2  # your limit (keep as-is)

for spec_name, predictors in VAR_SPECS.items():
    print(f"\n=== VAR order selection: {spec_name} ===")

    Z = build_var_dataset(uk_stationary, predictors)

    # Pre-2018 only for selecting lag order
    Z_train = Z.loc[Z.index < START_TEST].copy()

    T0, k0 = len(Z_train), Z_train.shape[1]
    if T0 < 10 or k0 < 2:
        # Extremely small / degenerate case
        print(f"[{spec_name}] Very small pre-test sample (T={T0}, k={k0}). Using p=1.")
        VAR_LAG_ORDERS[spec_name] = 1
        continue

    p_cap = feasible_maxlags(T0, k0, MAX_LAGS)

    # If even p=1 is risky, still fallback to 1
    if p_cap < 1:
        print(f"[{spec_name}] Not enough observations (T={T0}, k={k0}). Using p=1.")
        VAR_LAG_ORDERS[spec_name] = 1
        continue

    # Try select_order safely
    try:
        sel = VAR(Z_train).select_order(maxlags=p_cap)

        # Nice display (optional)
        try:
            print(sel.summary())
        except Exception:
            pass

        # selected_orders dict
        selected = getattr(sel, "selected_orders", {}) or {}

        p_opt = None
        for crit in ["aic", "hqic", "fpe", "bic"]:
            if crit in selected and selected[crit] is not None:
                p_opt = int(selected[crit])
                break

        # Final fallback: if for any reason nothing is selected
        if p_opt is None:
            p_opt = p_cap

    except Exception as e:
        print(f"[{spec_name}] Lag selection failed ({type(e).__name__}: {e}). Using p=1.")
        p_opt = 1

    # Clamp to [1, MAX_LAGS]
    p_opt = int(max(1, min(p_opt, MAX_LAGS)))
    VAR_LAG_ORDERS[spec_name] = p_opt

    print(f"Chosen lag length for {spec_name}: p = {p_opt}")



=== VAR order selection: traditional ===
 VAR Order Selection (* highlights the minimums) 
      AIC         BIC         FPE         HQIC   
-------------------------------------------------
0      -45.24     -44.80*   2.249e-20      -45.14
1     -46.88*      -42.42  8.337e-21*     -45.83*
-------------------------------------------------
Chosen lag length for traditional: p = 1

=== VAR order selection: highfreq ===
[highfreq] Lag selection failed (ValueError: maxlags is too large for the number of observations and the number of equations. The largest model cannot be estimated.). Using p=1.
Chosen lag length for highfreq: p = 1

=== VAR order selection: combined ===
[combined] Lag selection failed (ValueError: maxlags is too large for the number of observations and the number of equations. The largest model cannot be estimated.). Using p=1.
Chosen lag length for combined: p = 1


In [ ]:
# VAR- 3 specifications using an expanding window framework

import time

START_TEST = pd.Timestamp('2018-01-31').to_period('M').to_timestamp('M')

# Initializing var_nowcasts and results before the loop
var_nowcasts = {}
results = []

for spec_name, predictors in VAR_SPECS.items():
    print(f"\n=== Rolling VAR nowcast: {spec_name} spec ===")

    p_opt = VAR_LAG_ORDERS[spec_name]
    Z = build_var_dataset(uk_stationary, predictors)

    preds, actuals, dates = [], [], []
    train_times, pred_times = [], []

    t0 = time.time()

    # Expanding-window one-step-ahead forecasts
    for t in Z.index[Z.index >= START_TEST]:
        train_end = (t.to_period('M') - 1).to_timestamp('M')
        train = Z.loc[:train_end]

        # Require some history: at least p_opt + 24 months
        if len(train) < (p_opt + 24):
            continue

        try:
            # Estimate VAR and time the fit
            t_train_start = time.time()
            res = VAR(train).fit(p_opt)
            t_train_end = time.time()
            train_times.append(t_train_end - t_train_start)

            # One-step-ahead forecast and timing
            t_pred_start = time.time()
            yhat = res.forecast(train.values[-p_opt:], steps=1)[0]
            t_pred_end = time.time()
            pred_times.append(t_pred_end - t_pred_start)

            hpi_idx = list(train.columns).index('HPI_TRANS')
            preds.append(float(yhat[hpi_idx]))
            actuals.append(float(Z.loc[t, 'HPI_TRANS']))
            dates.append(t)

        except Exception:
            continue

    total_time = time.time() - t0

    # Accuracy
    rmse, mae = eval_metrics(actuals, preds)
    print(f"{spec_name}: Steps={len(preds)}  RMSE={rmse:.5f}  MAE={mae:.5f}")

    # Efficiency summaries
    avg_train = float(np.mean(train_times)) if train_times else np.nan
    avg_pred  = float(np.mean(pred_times)) if pred_times else np.nan
    avg_total = avg_train + avg_pred if train_times and pred_times else np.nan

    print("VAR computational efficiency:")
    print(f"  Total rolling nowcast time: {total_time:.2f} seconds")
    print(f"  Avg training time per forecast:   {avg_train:.4f} seconds")
    print(f"  Avg prediction time per forecast: {avg_pred:.6f} seconds")
    print(f"  Avg total time per forecast:      {avg_total:.4f} seconds")

    # Save prediction vs actual series
    nowcast_df = pd.DataFrame(
        {f"VAR_{spec_name}_pred": preds, "HPI_TRANS": actuals},
        index=pd.Index(dates, name="Date")
    )
    nowcast_df.to_csv(f"VAR_nowcast_{spec_name}.csv")
    print(f"Saved VAR_nowcast_{spec_name}.csv")

    var_nowcasts[spec_name] = nowcast_df
    results.append({
        "spec": spec_name,
        "p": p_opt,
        "rmse": rmse,
        "mae": mae,
        "total_time": total_time,
        "avg_train_time": avg_train,
        "avg_pred_time": avg_pred,
        "avg_total_time": avg_total
    })

# Building comparison table from results list
var_cmp = pd.DataFrame(results)
var_cmp = var_cmp.drop_duplicates(subset="spec", keep="last")
var_cmp = var_cmp.set_index("spec")

display(var_cmp)


=== Rolling VAR nowcast: traditional spec ===
traditional: Steps=88  RMSE=0.01689  MAE=0.01138
VAR computational efficiency:
  Total rolling nowcast time: 0.25 seconds
  Avg training time per forecast:   0.0023 seconds
  Avg prediction time per forecast: 0.000102 seconds
  Avg total time per forecast:      0.0024 seconds
Saved VAR_nowcast_traditional.csv

=== Rolling VAR nowcast: highfreq spec ===
highfreq: Steps=88  RMSE=0.01573  MAE=0.01107
VAR computational efficiency:
  Total rolling nowcast time: 0.25 seconds
  Avg training time per forecast:   0.0023 seconds
  Avg prediction time per forecast: 0.000103 seconds
  Avg total time per forecast:      0.0024 seconds
Saved VAR_nowcast_highfreq.csv

=== Rolling VAR nowcast: combined spec ===
combined: Steps=88  RMSE=0.01727  MAE=0.01198
VAR computational efficiency:
  Total rolling nowcast time: 0.28 seconds
  Avg training time per forecast:   0.0026 seconds
  Avg prediction time per forecast: 0.000132 seconds
  Avg total time per forec

,p,rmse,mae,total_time,avg_train_time,avg_pred_time,avg_total_time
spec,,,,,,,
traditional,1,0.016894,0.011382,0.247169,0.002264,0.000102,0.002366
highfreq,1,0.015735,0.011066,0.246845,0.002254,0.000103,0.002356
combined,1,0.017268,0.011977,0.282660,0.002635,0.000132,0.002767


Machine Learning Models

In [ ]:
# Defining the 60-month rolling window
def rolling_panel_splits(all_dates, start_test, window_months=60):
    """
    Generate (train_start, train_end) and test_date for a rolling, fixed-length window.
    - Train window: last `window_months` months before each test_date.
    - Test: one month ahead (test_date).
    """
    test_dates = [d for d in all_dates if d >= start_test]
    for test_date in test_dates:
        train_end = test_date - pd.offsets.MonthBegin(1)
        train_start = train_end - pd.DateOffset(months=window_months) + pd.offsets.MonthBegin(1)
        yield (train_start, train_end), test_date

Ridge Regression

In [ ]:
import time
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Loading the final_ml dataset
try:
    df = final_ml.copy()
except NameError:
    df = pd.read_csv("regional_nowcast_features_lag1to3.csv")

df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Date", "RegionName"]).reset_index(drop=True)

TARGET = "HPI_TRANS"
START_TEST = pd.Timestamp("2018-01-01")
WINDOW_MONTHS = 60
all_dates = sorted(df["Date"].unique())

spec_label = {
    "traditional": "Traditional",
    "highfreq": "High-Frequency",
    "combined": "Combined"
}

# Ridge model with scaling
ridge_model = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=1.0, random_state=42))
])

ridge_results = []
ridge_nowcasts = {}

print("\nRunning Ridge regression nowcasts (rolling 60-month window)...")

for spec_key, feature_cols in FEATURE_SETS.items():
    print(f"\n--- Ridge: {spec_label.get(spec_key, spec_key)} specification ---")

    # Checking if all features are present
    missing = [c for c in feature_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing lagged columns for '{spec_key}': {missing[:5]}")

    uk_true, uk_pred, dates_used = [], [], []
    train_times, pred_times = [], []

    t0 = time.time()

    # Expanding Rolling window framework
    for (train_start, train_end), test_date in rolling_panel_splits(
        all_dates, START_TEST, window_months=WINDOW_MONTHS
    ):
        tr_idx = (df["Date"] >= train_start) & (df["Date"] <= train_end)
        te_idx = (df["Date"] == test_date)

        train_df = df.loc[tr_idx]
        test_df  = df.loc[te_idx]

        # Require a minimum of 24 distinct months and at least one test row
        if train_df["Date"].nunique() < 24 or test_df.empty:
            continue

        X_tr, y_tr = train_df[feature_cols], train_df[TARGET]
        X_te, y_te = test_df[feature_cols], test_df[TARGET]

        # Time training
        t_train_start = time.time()
        ridge_model.fit(X_tr, y_tr)
        t_train_end = time.time()
        train_times.append(t_train_end - t_train_start)

        # Time prediction
        t_pred_start = time.time()
        preds_region = ridge_model.predict(X_te)
        t_pred_end = time.time()
        pred_times.append(t_pred_end - t_pred_start)

        # Aggregate to UK level (mean across regions)
        uk_pred.append(float(np.mean(preds_region)))
        uk_true.append(float(np.mean(y_te)))
        dates_used.append(test_date)

    total_time = time.time() - t0
    n_forecasts = len(uk_true)

    rmse, mae = eval_metrics(uk_true, uk_pred)
    print(f"Ridge [{spec_label.get(spec_key, spec_key)}] — RMSE={rmse:.5f}, MAE={mae:.5f}")

    if n_forecasts > 0:
        avg_train = float(np.mean(train_times))
        avg_pred  = float(np.mean(pred_times))
        avg_total = total_time / n_forecasts
    else:
        avg_train = avg_pred = avg_total = np.nan

    print("Computational efficiency:")
    print(f"  Total rolling-window time: {total_time:.2f} seconds")
    print(f"  Avg training time per forecast:   {avg_train:.6f} seconds")
    print(f"  Avg prediction time per forecast: {avg_pred:.6f} seconds")
    print(f"  Avg total time per forecast:      {avg_total:.6f} seconds")

    # Save UK-level nowcast
    nowcast_df = pd.DataFrame({
        "HPI_TRANS_actual_UK_mean": uk_true,
        "HPI_TRANS_pred_Ridge": uk_pred
    }, index=pd.to_datetime(dates_used))
    nowcast_df.index.name = "Date"

    fname = f"UK_nowcast_Ridge_{spec_key}_rolling{WINDOW_MONTHS}m.csv"
    nowcast_df.to_csv(fname)
    print(f"Saved: {fname}")

    ridge_nowcasts[spec_key] = nowcast_df

    ridge_results.append({
        "Model": "Ridge",
        "Specification": spec_label.get(spec_key, spec_key),
        "Window": f"Rolling {WINDOW_MONTHS} months",
        "RMSE": rmse,
        "MAE": mae,
        "TotalTime_sec": total_time,
        "AvgTrain_sec": avg_train,
        "AvgPred_sec": avg_pred,
        "AvgTotal_sec": avg_total,
        "n_forecasts": n_forecasts
    })

ridge_summary = pd.DataFrame(ridge_results).sort_values("RMSE").reset_index(drop=True)
display(ridge_summary)

ridge_summary.to_csv(f"Ridge_Evaluation_Summary_Rolling{WINDOW_MONTHS}m.csv", index=False)
print("\nSaved summary → Ridge_Evaluation_Summary_Rolling60m.csv")


Running Ridge regression nowcasts (rolling 60-month window)...

--- Ridge: Traditional specification ---
Ridge [Traditional] — RMSE=0.00972, MAE=0.00682
Computational efficiency:
  Total rolling-window time: 3.02 seconds
  Avg training time per forecast:   0.020722 seconds
  Avg prediction time per forecast: 0.002693 seconds
  Avg total time per forecast:      0.035103 seconds
Saved: UK_nowcast_Ridge_traditional_rolling60m.csv

--- Ridge: High-Frequency specification ---
Ridge [High-Frequency] — RMSE=0.00618, MAE=0.00511
Computational efficiency:
  Total rolling-window time: 3.48 seconds
  Avg training time per forecast:   0.024679 seconds
  Avg prediction time per forecast: 0.002984 seconds
  Avg total time per forecast:      0.040408 seconds
Saved: UK_nowcast_Ridge_highfreq_rolling60m.csv

--- Ridge: Combined specification ---
Ridge [Combined] — RMSE=0.01707, MAE=0.00901
Computational efficiency:
  Total rolling-window time: 3.21 seconds
  Avg training time per forecast:   0.023762 

,Model,Specification,Window,RMSE,MAE,TotalTime_sec,AvgTrain_sec,AvgPred_sec,AvgTotal_sec,n_forecasts
0,Ridge,High-Frequency,Rolling 60 months,0.006180,0.005110,3.475080,0.024679,0.002984,0.040408,86
1,Ridge,Traditional,Rolling 60 months,0.009717,0.006819,3.018852,0.020722,0.002693,0.035103,86
2,Ridge,Combined,Rolling 60 months,0.017071,0.009011,3.211952,0.023762,0.002675,0.037348,86



Saved summary → Ridge_Evaluation_Summary_Rolling60m.csv


Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso

lasso_model = Pipeline([
    ("scaler", StandardScaler()),
    ("lasso", Lasso(alpha=0.0005, random_state=42, max_iter=10000))
])

lasso_results = []
lasso_nowcasts = {}

print("\nRunning Lasso regression nowcasts (rolling 60-month window)...")

for spec_key, feature_cols in FEATURE_SETS.items():
    print(f"\n--- Lasso: {spec_label.get(spec_key, spec_key)} specification ---")

    missing = [c for c in feature_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing lagged columns for '{spec_key}': {missing[:5]}")

    uk_true, uk_pred, dates_used = [], [], []
    train_times, pred_times = [], []

    t0 = time.time()

 # Expanding Rolling window framework
    for (train_start, train_end), test_date in rolling_panel_splits(
        all_dates, START_TEST, window_months=WINDOW_MONTHS
    ):
        tr_idx = (df["Date"] >= train_start) & (df["Date"] <= train_end)
        te_idx = (df["Date"] == test_date)

        train_df = df.loc[tr_idx]
        test_df  = df.loc[te_idx]

        if train_df["Date"].nunique() < 24 or test_df.empty:
            continue

        X_tr, y_tr = train_df[feature_cols], train_df[TARGET]
        X_te, y_te = test_df[feature_cols], test_df[TARGET]

        # Time training
        t_train_start = time.time()
        lasso_model.fit(X_tr, y_tr)
        t_train_end = time.time()
        train_times.append(t_train_end - t_train_start)

        # Time prediction
        t_pred_start = time.time()
        preds_region = lasso_model.predict(X_te)
        t_pred_end = time.time()
        pred_times.append(t_pred_end - t_pred_start)

        uk_pred.append(float(np.mean(preds_region)))
        uk_true.append(float(np.mean(y_te)))
        dates_used.append(test_date)

    total_time = time.time() - t0
    n_forecasts = len(uk_true)

    rmse, mae = eval_metrics(uk_true, uk_pred)
    print(f"Lasso [{spec_label.get(spec_key, spec_key)}] — RMSE={rmse:.5f}, MAE={mae:.5f}")

    if n_forecasts > 0:
        avg_train = float(np.mean(train_times))
        avg_pred  = float(np.mean(pred_times))
        avg_total = total_time / n_forecasts
    else:
        avg_train = avg_pred = avg_total = np.nan

    print("Computational efficiency:")
    print(f"  Total rolling-window time: {total_time:.2f} seconds")
    print(f"  Avg training time per forecast:   {avg_train:.6f} seconds")
    print(f"  Avg prediction time per forecast: {avg_pred:.6f} seconds")
    print(f"  Avg total time per forecast:      {avg_total:.6f} seconds")

    # Save UK-level nowcast
    nowcast_df = pd.DataFrame({
        "HPI_TRANS_actual_UK_mean": uk_true,
        "HPI_TRANS_pred_Lasso": uk_pred
    }, index=pd.to_datetime(dates_used))
    nowcast_df.index.name = "Date"

    fname = f"UK_nowcast_Lasso_{spec_key}_rolling{WINDOW_MONTHS}m.csv"
    nowcast_df.to_csv(fname)
    print(f"Saved: {fname}")

    lasso_nowcasts[spec_key] = nowcast_df

    lasso_results.append({
        "Model": "Lasso",
        "Specification": spec_label.get(spec_key, spec_key),
        "Window": f"Rolling {WINDOW_MONTHS} months",
        "RMSE": rmse,
        "MAE": mae,
        "TotalTime_sec": total_time,
        "AvgTrain_sec": avg_train,
        "AvgPred_sec": avg_pred,
        "AvgTotal_sec": avg_total,
        "n_forecasts": n_forecasts
    })

lasso_summary = pd.DataFrame(lasso_results).sort_values("RMSE").reset_index(drop=True)
display(lasso_summary)

lasso_summary.to_csv(f"Lasso_Evaluation_Summary_Rolling{WINDOW_MONTHS}m.csv", index=False)
print("\nSaved summary → Lasso_Evaluation_Summary_Rolling60m.csv")


Running Lasso regression nowcasts (rolling 60-month window)...

--- Lasso: Traditional specification ---
Lasso [Traditional] — RMSE=0.00519, MAE=0.00427
Computational efficiency:
  Total rolling-window time: 7.60 seconds
  Avg training time per forecast:   0.065478 seconds
  Avg prediction time per forecast: 0.004969 seconds
  Avg total time per forecast:      0.088319 seconds
Saved: UK_nowcast_Lasso_traditional_rolling60m.csv

--- Lasso: High-Frequency specification ---
Lasso [High-Frequency] — RMSE=0.00531, MAE=0.00441
Computational efficiency:
  Total rolling-window time: 3.71 seconds
  Avg training time per forecast:   0.028181 seconds
  Avg prediction time per forecast: 0.002879 seconds
  Avg total time per forecast:      0.043096 seconds
Saved: UK_nowcast_Lasso_highfreq_rolling60m.csv

--- Lasso: Combined specification ---
Lasso [Combined] — RMSE=0.00521, MAE=0.00437
Computational efficiency:
  Total rolling-window time: 6.23 seconds
  Avg training time per forecast:   0.055932 

,Model,Specification,Window,RMSE,MAE,TotalTime_sec,AvgTrain_sec,AvgPred_sec,AvgTotal_sec,n_forecasts
0,Lasso,Traditional,Rolling 60 months,0.005189,0.004269,7.595395,0.065478,0.004969,0.088319,86
1,Lasso,Combined,Rolling 60 months,0.005210,0.004371,6.225457,0.055932,0.003388,0.072389,86
2,Lasso,High-Frequency,Rolling 60 months,0.005313,0.004407,3.706214,0.028181,0.002879,0.043096,86



Saved summary → Lasso_Evaluation_Summary_Rolling60m.csv


XGBoost

In [ ]:
from xgboost import XGBRegressor

# Loading the final_ml dataset
try:
    df = final_ml.copy()
except NameError:
    df = pd.read_csv("regional_nowcast_features_lag1to3.csv")

df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Date", "RegionName"]).reset_index(drop=True)

TARGET = "HPI_TRANS"

spec_label = {
    "traditional": "Traditional",
    "highfreq": "High-Frequency",
    "combined": "Combined",
}

all_dates     = sorted(df["Date"].unique())
START_TEST    = pd.Timestamp("2018-01-01")
WINDOW_MONTHS = 60

xgb_results  = []
xgb_nowcasts = {}

# Loop over each XGBoost specification
for spec_key, feature_cols in FEATURE_SETS.items():
    print(f"\n=== XGBoost — {spec_label.get(spec_key, spec_key)} specification ===")

    # Checking that all required features exist
    missing = [c for c in feature_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing lagged features for spec '{spec_key}': {missing[:5]} ...")

    # XGBoost configuration for the three specifications
    xgb_model = XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1,
    )

    uk_true, uk_pred, dates_used = [], [], []
    train_times, pred_times      = [], []

    t_total_start = time.time()

    # Rolling 60-month window framework
    for (train_start, train_end), test_date in rolling_panel_splits(
        all_dates, start_test=START_TEST, window_months=WINDOW_MONTHS
    ):
        tr_mask = (df["Date"] >= train_start) & (df["Date"] <= train_end)
        te_mask = (df["Date"] == test_date)

        train_df = df.loc[tr_mask]
        test_df  = df.loc[te_mask]

        # Require enough history and a non-empty test month
        if train_df["Date"].nunique() < 24 or test_df.empty:
            continue

        X_tr, y_tr = train_df[feature_cols], train_df[TARGET]
        X_te, y_te = test_df[feature_cols], test_df[TARGET]

        # Measure training time
        t_fit_start = time.time()
        xgb_model.fit(X_tr, y_tr)
        t_fit_end   = time.time()
        train_times.append(t_fit_end - t_fit_start)

        # Measure prediction time
        t_pred_start = time.time()
        preds_region = xgb_model.predict(X_te)
        t_pred_end   = time.time()
        pred_times.append(t_pred_end - t_pred_start)

        # Aggregate regional predictions to UK level
        uk_pred.append(float(np.mean(preds_region)))
        uk_true.append(float(np.mean(y_te.values)))
        dates_used.append(test_date)

    t_total_end = time.time()
    total_time  = t_total_end - t_total_start
    n_forecasts = len(uk_true)

    # Evaluation metrics for Forecast accuracy
    rmse, mae = eval_metrics(uk_true, uk_pred)
    print(f"XGBoost [{spec_label.get(spec_key, spec_key)}] — RMSE={rmse:.5f}, MAE={mae:.5f}")

    # Computational efficiency
    if n_forecasts > 0:
        avg_train = float(np.mean(train_times))
        avg_pred  = float(np.mean(pred_times))
        avg_total = float(total_time / n_forecasts)
    else:
        avg_train = avg_pred = avg_total = np.nan

    print(f"XGBoost computational efficiency [{spec_label.get(spec_key, spec_key)}]:")
    print(f"  Total rolling-window time:       {total_time:.2f} seconds")
    print(f"  Avg training time per forecast:  {avg_train:.6f} seconds")
    print(f"  Avg prediction time per forecast:{avg_pred:.6f} seconds")
    print(f"  Avg total time per forecast:     {avg_total:.6f} seconds")

    xgb_results.append({
        "Model": "XGBoost",
        "Specification": spec_label.get(spec_key, spec_key),
        "Window": f"Rolling-{WINDOW_MONTHS}m",
        "RMSE": rmse,
        "MAE": mae,
        "TotalTime_sec": total_time,
        "AvgTrain_sec": avg_train,
        "AvgPred_sec": avg_pred,
        "AvgTotal_sec": avg_total,
        "n_forecasts": n_forecasts,
    })

    # Saving UK-level nowcast series
    nowcast_df = pd.DataFrame({
        "HPI_TRANS_actual_UK_mean": uk_true,
        "HPI_TRANS_pred_XGB": uk_pred,
    }, index=pd.to_datetime(dates_used))
    nowcast_df.index.name = "Date"

    fname = f"UK_nowcast_XGB_{spec_key}_rolling{WINDOW_MONTHS}m.csv"
    nowcast_df.to_csv(fname)
    print(f"Saved: {fname}")

    xgb_nowcasts[spec_key] = nowcast_df

# Summary table for the three specifications
xgb_summary = (
    pd.DataFrame(xgb_results)
    .sort_values("RMSE")
    .reset_index(drop=True)
)

display(xgb_summary)

summary_name = f"XGBoost_Evaluation_Summary_Rolling{WINDOW_MONTHS}m.csv"
xgb_summary.to_csv(summary_name, index=False)
print(f"Saved evaluation summary: {summary_name}")


=== XGBoost — Traditional specification ===
XGBoost [Traditional] — RMSE=0.00516, MAE=0.00416
XGBoost computational efficiency [Traditional]:
  Total rolling-window time:       173.67 seconds
  Avg training time per forecast:  1.994648 seconds
  Avg prediction time per forecast:0.012855 seconds
  Avg total time per forecast:     2.019398 seconds
Saved: UK_nowcast_XGB_traditional_rolling60m.csv

=== XGBoost — High-Frequency specification ===
XGBoost [High-Frequency] — RMSE=0.00595, MAE=0.00482
XGBoost computational efficiency [High-Frequency]:
  Total rolling-window time:       85.41 seconds
  Avg training time per forecast:  0.974108 seconds
  Avg prediction time per forecast:0.009562 seconds
  Avg total time per forecast:     0.993135 seconds
Saved: UK_nowcast_XGB_highfreq_rolling60m.csv

=== XGBoost — Combined specification ===
XGBoost [Combined] — RMSE=0.00518, MAE=0.00416
XGBoost computational efficiency [Combined]:
  Total rolling-window time:       163.51 seconds
  Avg training 

,Model,Specification,Window,RMSE,MAE,TotalTime_sec,AvgTrain_sec,AvgPred_sec,AvgTotal_sec,n_forecasts
0,XGBoost,Traditional,Rolling-60m,0.005159,0.004161,173.668247,1.994648,0.012855,2.019398,86
1,XGBoost,Combined,Rolling-60m,0.005178,0.004156,163.507052,1.878575,0.011786,1.901245,86
2,XGBoost,High-Frequency,Rolling-60m,0.005955,0.004820,85.409586,0.974108,0.009562,0.993135,86


Saved evaluation summary: XGBoost_Evaluation_Summary_Rolling60m.csv


Random Forest

In [ ]:
# Random Forest

from sklearn.ensemble import RandomForestRegressor

# Loading the final_ml dataset
try:
    df = final_ml.copy()
except NameError:
    df = pd.read_csv("regional_nowcast_features_lag1to3.csv")

df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Date", "RegionName"]).reset_index(drop=True)

TARGET = "HPI_TRANS"
all_dates = sorted(df["Date"].unique())

START_TEST    = pd.Timestamp("2018-01-01")
WINDOW_MONTHS = 60

rf_results   = []
rf_nowcasts  = {}
spec_labels  = {"traditional": "Traditional",
                "highfreq": "High-Frequency",
                "combined": "Combined"}

for spec_key, feature_cols in FEATURE_SETS.items():
    print(f"\n=== Random Forest — {spec_labels.get(spec_key, spec_key)} specification ===")

    # checking that all columns present
    missing = [c for c in feature_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing features for '{spec_key}': {missing[:5]}...")

    rf = RandomForestRegressor(
        n_estimators=600,
        max_depth=None,
        min_samples_leaf=3,
        max_features="sqrt",
        random_state=42,
        n_jobs=-1
    )

    uk_true, uk_pred, dates_used = [], [], []
    train_times, pred_times = [], []

    t0 = time.time()

    for (train_start, train_end), test_date in rolling_panel_splits(
        all_dates, start_test=START_TEST, window_months=WINDOW_MONTHS
    ):
        tr_mask = (df["Date"] >= train_start) & (df["Date"] <= train_end)
        te_mask = (df["Date"] == test_date)

        train_df = df.loc[tr_mask]
        test_df  = df.loc[te_mask]

        # Need at least 24 distinct months in training and a non-empty test month
        if train_df["Date"].nunique() < 24 or test_df.empty:
            continue

        X_tr, y_tr = train_df[feature_cols], train_df[TARGET]
        X_te, y_te = test_df[feature_cols], test_df[TARGET]

        # Timed training
        t_train_start = time.time()
        rf.fit(X_tr, y_tr)
        t_train_end = time.time()
        train_times.append(t_train_end - t_train_start)

        # Timed prediction
        t_pred_start = time.time()
        preds_region = rf.predict(X_te)
        t_pred_end = time.time()
        pred_times.append(t_pred_end - t_pred_start)

        # Aggregate from regions to UK
        uk_pred.append(float(np.mean(preds_region)))
        uk_true.append(float(np.mean(y_te.values)))
        dates_used.append(test_date)

    total_time = time.time() - t0
    n_forecasts = len(uk_true)

    # Evaluation metrics for Forecast Accuracy
    rmse, mae = eval_metrics(uk_true, uk_pred)
    print(f"Random Forest [{spec_labels.get(spec_key, spec_key)}] — RMSE={rmse:.5f}, MAE={mae:.5f}")

    # Computational efficiency
    if n_forecasts > 0:
        avg_train = float(np.mean(train_times))
        avg_pred  = float(np.mean(pred_times))
        avg_total = float(total_time / n_forecasts)
    else:
        avg_train = avg_pred = avg_total = np.nan

    print("Computational efficiency:")
    print(f"  Total rolling-window time:       {total_time:.2f} sec")
    print(f"  Avg training time per forecast:  {avg_train:.6f} sec")
    print(f"  Avg prediction time per forecast:{avg_pred:.6f} sec")
    print(f"  Avg total time per forecast:     {avg_total:.6f} sec")

    # Save UK-level nowcast
    nowcast_df = pd.DataFrame({
        "HPI_TRANS_actual_UK_mean": uk_true,
        "HPI_TRANS_pred_RF": uk_pred
    }, index=pd.to_datetime(dates_used))
    nowcast_df.index.name = "Date"

    fname = f"UK_nowcast_RF_{spec_labels.get(spec_key, spec_key)}_rolling{WINDOW_MONTHS}m.csv"
    nowcast_df.to_csv(fname)
    print(f"Saved: {fname}")

    rf_nowcasts[spec_key] = nowcast_df

    rf_results.append({
        "Model": "Random Forest",
        "Specification": spec_labels.get(spec_key, spec_key),
        "Window": f"Rolling-{WINDOW_MONTHS}m",
        "RMSE": rmse,
        "MAE": mae,
        "TotalTime_sec": total_time,
        "AvgTrain_sec": avg_train,
        "AvgPred_sec": avg_pred,
        "AvgTotal_sec": avg_total,
        "n_forecasts": n_forecasts
    })

# Summary table
rf_summary = pd.DataFrame(rf_results).sort_values("RMSE").reset_index(drop=True)
display(rf_summary)

rf_summary.to_csv(f"RF_Evaluation_Summary_Rolling{WINDOW_MONTHS}m.csv", index=False)
print(f"Saved: RF_Evaluation_Summary_Rolling{WINDOW_MONTHS}m.csv")


=== Random Forest — Traditional specification ===
Random Forest [Traditional] — RMSE=0.00497, MAE=0.00411
Computational efficiency:
  Total rolling-window time:       471.25 sec
  Avg training time per forecast:  5.279054 sec
  Avg prediction time per forecast:0.192188 sec
  Avg total time per forecast:     5.479680 sec
Saved: UK_nowcast_RF_Traditional_rolling60m.csv

=== Random Forest — High-Frequency specification ===
Random Forest [High-Frequency] — RMSE=0.00553, MAE=0.00446
Computational efficiency:
  Total rolling-window time:       507.94 sec
  Avg training time per forecast:  5.708852 sec
  Avg prediction time per forecast:0.189271 sec
  Avg total time per forecast:     5.906259 sec
Saved: UK_nowcast_RF_High-Frequency_rolling60m.csv

=== Random Forest — Combined specification ===
Random Forest [Combined] — RMSE=0.00493, MAE=0.00404
Computational efficiency:
  Total rolling-window time:       643.95 sec
  Avg training time per forecast:  7.289457 sec
  Avg prediction time per fo

,Model,Specification,Window,RMSE,MAE,TotalTime_sec,AvgTrain_sec,AvgPred_sec,AvgTotal_sec,n_forecasts
0,Random Forest,Combined,Rolling-60m,0.004926,0.004043,643.953659,7.289457,0.188951,7.487833,86
1,Random Forest,Traditional,Rolling-60m,0.004965,0.004108,471.252498,5.279054,0.192188,5.479680,86
2,Random Forest,High-Frequency,Rolling-60m,0.005527,0.004458,507.938237,5.708852,0.189271,5.906259,86


Saved: RF_Evaluation_Summary_Rolling60m.csv
